In [1]:
import tensorflow as tf
import cv2
import numpy as np
import makiflow as mf
mf.set_main_gpu(1)
import matplotlib.pyplot as plt
from makiflow.layers import InputLayer
from pose_estimation.model import PEModel

from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import matplotlib.pyplot as plt

import numpy as np
import skimage.io as io

%matplotlib inline

/home/ikilbas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ikilbas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ikilbas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ikilbas/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

## Create and init model

In [ ]:
W, H = (300, 300)

In [2]:
in_x = InputLayer([1, W, H, 3], name='in_x')
model = PEModel.from_json('model_mobilev2_arch_300.json', in_x)
sess = tf.Session()
model.set_session(sess)
model.load_weights('/mnt/data/pose_estimation_coco/experiments/danil_2_test_mobile_300/last_weights/weights.ckpt')



Model is restored!



Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from /mnt/data/pose_estimation_coco/experiments/danil_2_test_mobile_300/last_weights/weights.ckpt
Weights are loaded.


## Re-layout COCO json and create json from predictions

In [ ]:
from pose_estimation.metrics.COCO_WholeBody import create_prediction_coco_json, relayout_keypoints

In [ ]:
# Original annotation file
annFile = '/mnt/data/coco_keypoints/coco_wholebody_val_v1.0.json'

In [5]:
relayout_keypoints(W, H, annFile, 'relayout_300_coco_whole_body_val.json')

loading annotations into memory...
Done (t=0.72s)
creating index...
index created!


 98%|█████████▊| 10777/11004 [00:07<00:00, 1414.88it/s]


In [6]:
create_prediction_coco_json(W, H, model,
                            'relayout_300_coco_whole_body_val.json',
                            'predicted_300_coco_whole_body_val.json',
                           '/mnt/data/coco_keypoints/val2017')

loading annotations into memory...


  0%|          | 0/5000 [00:00<?, ?it/s]

Done (t=1.00s)
creating index...
index created!


100%|██████████| 5000/5000 [03:45<00:00, 22.19it/s]


## Estimate with coco

In [7]:
from pose_estimation.metrics.COCO_WholeBody import MYeval_wholebody

In [8]:
cocoGt = COCO('relayout_300_coco_whole_body_val.json')

loading annotations into memory...
Done (t=0.73s)
creating index...
index created!


In [9]:
cocoDt = cocoGt.loadRes('predicted_300_coco_whole_body_val.json')

Loading and preparing results...
DONE (t=0.13s)
creating index...
index created!


In [10]:
cocoEval = MYeval_wholebody(cocoDt=cocoDt, cocoGt=cocoGt)

In [11]:
cocoEval.evaluate()

Running per image evaluation...
DONE (t=1.34s).


In [12]:
cocoEval.accumulate()

Accumulating evaluation results...
DONE (t=0.06s).


In [13]:
cocoEval.summarize()

 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.026
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.089
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.008
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.043
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.062
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 20 ] = 0.056
 Average Recall     (AR) @[ IoU=0.50      | area=   all | maxDets= 20 ] = 0.152
 Average Recall     (AR) @[ IoU=0.75      | area=   all | maxDets= 20 ] = 0.032
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets= 20 ] = 0.044
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets= 20 ] = 0.174
